## Note

This works need to be adapted for TSN network
- Replace $o_i$ as traverse every instance from $lcm(i, j) / p_j$

In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from gurobipy import GRB
import gurobipy as gp

In [48]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 100000
DATA_NAME = "harmonic0"
TOPO_NAME = "2"

task = pd.read_csv("../../rtns_data/utilization/utilization_25_41.csv")
network = pd.read_csv("../../rtns_data/utilization/utilization_topology.csv")

# task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
# network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

In [51]:
m = gp.Model("RTNS2017")

In [52]:
M = int(1e16)

Network model

$I=(V, E, D)$

In [53]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['msd'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

In [54]:
LCM = int(np.lcm.reduce(task['period']))

Task model

$$f_{k}=\left(s_{k}, d_{k}, c t_{k}, r s l_{k}, m l_{k}\right)$$

In [55]:
task_attr = {}

x = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.BINARY , name="routing")
t = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.INTEGER , name="time_start")


$x_{i j k}=\left\{\begin{array}{l}
1: \text { flow } f_{k} \text { is routed via link }(i, j) \\
0: \text { flow } f_{k} \text { does not use link }(i, j)
\end{array}\right.$

$t_{ijk}$: beginning of the time slot of flow f_k on link

$o_{ijk}$: a positive integer variable that states an offset of a time slot as a number of full cycles of length ct k

In [56]:
for k, row in task.iterrows():
    task_attr.setdefault(k, {})
    
    
    ## f_k
    task_attr[k]['s'] = int(row['src'])
    task_attr[k]['d'] = int(eval(row['dst'])[0])
    task_attr[k]['ct'] = int(row['period'])
    task_attr[k]['rsl'] = int(row['size'] * 8)
    task_attr[k]['ml'] = int(row['deadline'])
    
    ## Bound the t matrix
    for j in range(len(link_to_index)):
        m.addConstr(0 <= t[k][j])
        m.addConstr(t[k][j] <= task_attr[k]['ct'] - task_attr[k]['rsl'])


## 2. Constraints

### 2.1 Routing

Acting nodes: Exactly one more outgoing than incoming link

\begin{aligned}
&\forall f_{k} \in F: \\
&\qquad \sum_{j \in V \mid\left(s_{k}, j\right) \in E} x_{s_{k} j k}-\sum_{j \in V \mid\left(j, s_{k}\right) \in E} x_{j s_{k} k}=1
\end{aligned}

In [57]:
link_in = {}
link_out = {}
for link in link_to_index.keys():
    link = eval(link)
    link_in.setdefault(link[1], [])
    link_in[link[1]].append(str(link))
    link_out.setdefault(link[0], [])
    link_out[link[0]].append(str(link))

In [58]:
for k in task_attr:
    m.addConstr(
        gp.quicksum(x[k][link_to_index[link]] 
                    for link in link_out[task_attr[k]['s']]) - 
        gp.quicksum(x[k][link_to_index[link]] 
                    for link in link_in[task_attr[k]['s']]) == 1
    )
    # m.addConstr(
    #     gp.quicksum(x[k][link_to_index[link]] 
    #                 for link in link_in[task_attr[k]['d']]) - 
    #     gp.quicksum(x[k][link_to_index[link]] 
    #                 for link in link_out[task_attr[k]['d']]) == 1
    # )

    # m.addConstr(
    #         gp.quicksum(x[k][link_to_index[link]] 
    #             for v in ES_set for link in link_in[v] ) == 1
    # )
    # m.addConstr(
    #         gp.quicksum(x[k][link_to_index[link]] 
    #             for v in ES_set for link in link_out[v]) == 1
    # )

Forwarding behavior

\begin{aligned}
&\forall f_{k} \in F, \forall i \in V \backslash\left\{s_{k}, d_{k}\right\}: \\
&\sum_{j \in V \mid(i, j) \in E} x_{i j k}-\sum_{j \in V \mid(j, i) \in E} x_{j i k}=0
\end{aligned}

In [59]:
for k in task_attr:
    for i in set(NODE_SET) - set([task_attr[k]['s'], task_attr[k]['d']]):
        m.addConstr(
            gp.quicksum(x[k][link_to_index[link]] for link in link_out[i])
            - 
            gp.quicksum(x[k][link_to_index[link]] for link in link_in[i])
            == 0
        )

Prune loop

\begin{aligned}
&\forall f_{k} \in F, \forall i \in V: \\
&\sum_{j \in V \mid(i, j) \in E} x_{i j k} \leq 1
\end{aligned}

In [60]:
for k in task_attr:
    for i in NODE_SET:
        m.addConstr(
            gp.quicksum(x[k][link_to_index[link]] for link in link_out[i]) <= 1
        )

Make schedule to zero when no path go through:

\begin{aligned}
&\forall f_{k} \in F, \forall(i, j) \in E: \\
&\qquad t_{i j k}+o_{i j k} \leq M * x_{i j k}
\end{aligned}

In [61]:
for k in task_attr:
    for link_index in index_to_link:
        m.addConstr(t[k][link_index] <=  M * x[k][link_index])

scheduling constraints that have to be met along a route from s_k to d_k

(packet can only be transmitted after fully procerssing)

\begin{aligned}
&\forall f_{k} \in F, \forall i \in V \backslash\left\{s_{k}, d_{k}\right\}: \\
&\sum_{j \in V \mid(i, j) \in E}\left(t_{i j k}+o_{i j k} * c t_{k}\right)-\sum_{j \in V \mid(j, i) \in E}\left(t_{j i k}+o_{j i k} * c t_{k}\right) \\
&\geq m s d_{i} * \sum_{j \in V \mid(i, j) \in E} x_{i j k}
\end{aligned}

In [62]:
for k in task_attr:
    for i in set(NODE_SET) - set([task_attr[k]['s'], task_attr[k]['d']]):
        m.addConstr(
            gp.quicksum(t[k][link_to_index[link]] for link in link_out[i])
            - 
            gp.quicksum(t[k][link_to_index[link]] for link in link_in[i])
            >= 
            (net_var[i]['msd'] + task_attr[k]['rsl']) * gp.quicksum(x[k][link_to_index[link]] for link in link_out[i])
        )

### 2.2 Resource constraints

A schedule is only feasible if the time slots on every link do not overlap.

\begin{gathered}
\forall\left(f_{k}, f_{l}\right) \in F \times F \mid l>k, \forall(i, j) \in E, \\
\forall(u, v) \in\left\{u \in \mathbb{N} \mid u \leq \frac{l c m\left(c t_{k}, c t_{l}\right)}{c t_{k}}\right\} \times\left\{v \in \mathbb{N} \mid v \leq \frac{l c m\left(c t_{k}, c t_{l}\right)}{c t_{l}}\right\}: \\
\left(t_{i j l}+v * c t_{l}\right)-\left(t_{i j k}+u * c t_{k}\right) \\
\geq r s l_{k}-M *\left(3-a_{i j k l u v}-x_{i j k}-x_{i j l}\right) \\
\quad\left(t_{i j k}+u * c t_{k}\right)-\left(t_{i j l}+v * c t_{l}\right) \\
\geq r s l_{l}-M *\left(2+a_{i j k l u v}-x_{i j k}-x_{i j l}\right)
\end{gathered}

In [63]:
for link in tqdm(link_to_index):
    link_i = link_to_index[link]
    for k, l in [(k, l) for k in task_attr for l in task_attr if k < l]:
        ctl, ctk = int(task_attr[l]['ct']), int(task_attr[k]['ct'])
        t_ijl, t_ijk = t[l][link_i], t[k][link_i]
        rsl_k, rsl_l = task_attr[k]['rsl'], task_attr[l]['rsl']
        for u, v in [(u, v) 
                     for u in range(0, int(np.lcm(ctk, ctl) / ctk)) 
                     for v in range(0, int(np.lcm(ctk, ctl) / ctl))]:
            _inte = m.addVar(vtype= GRB.BINARY, name = "%s%d%d%d%d"%(link, k, l, u, v))
            m.addConstr((t_ijl + v * ctl) - (t_ijk + u * ctk) 
                        >= rsl_k - M * (3 - _inte - x[k][link_i] - x[l][link_i]))
            m.addConstr((t_ijk + u * ctk) - (t_ijl + v * ctl)
                       >= rsl_l - M * (2 + _inte - x[k][link_i] - x[l][link_i]))

100%|██████████| 30/30 [00:11<00:00,  2.60it/s]


### 2.3 Application constraints

\begin{aligned}
&\forall f_{k} \in F: \\
&\sum_{\substack{j \in V \mid \\
\left(j, d_{k}\right) \in E}}\left(t_{j d_{k} k}+o_{j d_{k} k} * c t_{k}\right)-\sum_{\substack{j \in V \mid \\
\left(s_{k}, j\right) \in E}}\left(t_{s_{k} j k}+o_{s_{k} j k} * c t_{k}\right) \\
&\leq m l_{k}-r s l_{k}
\end{aligned}

In [64]:
for k in task_attr:
    m.addConstr(gp.quicksum(t[k][link_to_index[link]]
                            for link in link_in[task_attr[k]['d']])
               -
                gp.quicksum(t[k][link_to_index[link]] 
                            for link in link_out[task_attr[k]['s']])
                <= task_attr[k]['ml'] - task_attr[k]['rsl'] - net_var[i]['msd']
                # <= task_attr[k]['ml'] - task_attr[k]['rsl']
               )

## 3. Objective

\begin{aligned}
\min \sum_{f_{k} \in F} &\left(\sum_{j \in V \mid\left(j, d_{k}\right) \in E}\left(t_{j d_{k} k}+o_{j d_{k}} k * c t_{k}\right)\right.\\
&\left.-\sum_{j \in V \mid\left(s_{k}, j\right) \in E}\left(t_{s_{k} j k}+o_{s_{k} j k} * c t_{k}\right)\right)
\end{aligned}

In [65]:
# m.setObjective(
#     gp.quicksum(
#         gp.quicksum(t[k][link_to_index[link]] 
#                     for j in range(0, NUM_NODE) for link in link_out[j] if eval(link)[1] == task_attr[k]['d'])
#                    -
#         gp.quicksum(t[k][link_to_index[link]]
#                     for j in range(0, NUM_NODE) for link in link_in[j] if eval(link)[0] == task_attr[k]['s'])
#     for k in task_attr
#     )
#     ,GRB.MINIMIZE)

In [66]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Set parameter Threads to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 1 threads
Optimize a model with 346524 rows, 172710 columns and 1705986 nonzeros
Model fingerprint: 0x838a7c47
Variable types: 0 continuous, 172710 integer (170640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+16]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+16]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 316079 rows and 157264 columns (presolve time = 5s) ...
Presolve removed 329633 rows and 164097 columns
Presolve time: 6.39s
Presolved: 16891 rows, 8613 columns, 55744 nonzeros
Variable types: 0 continuous, 8613 integer (8159 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.228159e+04   0.000000e+00      7s
  

In [67]:
run_time = m.Runtime

In [68]:
run_time

8.873753070831299

In [19]:
queue_count = {}
queue_log = {}

## GCL
GCL = []
for i in task_attr:
    period = task.loc[i, 'period']
    for e_i in index_to_link:
        e = index_to_link[e_i]
        if x[i][e_i].x == 1:
            queue_count.setdefault(e, 0)
            start = t[i][e_i].x
            end = start + task_attr[i]['rsl']
            queue = queue_count[e]
            for k in range(int(LCM / period)):
                GCL.append(
                    [eval(e), queue, int(start + k * period) * macrotick, int(end + k * period) * macrotick, LCM * macrotick]
                )
            queue_log[(i, e)] = queue
            queue_count[e] += 1

In [20]:
## Offset
OFFSET = []
for i in task_attr:
    start_link = [link for link in link_out[task_attr[i]['s']] if x[i][link_to_index[link]].x == 1][0]
    offset = t[i, link_to_index[start_link]].x
    OFFSET.append(
        [i, 0, (task.loc[i,'period'] - offset) * macrotick]
    )    

In [21]:
ROUTE = []
for i in task_attr:
    for k, rr in enumerate(x[i]):
        if rr.x == 1:
            ROUTE.append(
                [i, eval(index_to_link[k])]
            )

In [22]:
QUEUE = []
for i in task_attr:
    for k, rr in enumerate(x[i]):
        if rr.x == 1:
            e = index_to_link[k]
            QUEUE.append([i, 0, eval(e), queue_log[(i, e)]])

In [23]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTNS2017-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTNS2017-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTNS2017-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTNS2017-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [24]:
GCL

,link,queue,start,end,cycle
0,"(3, 11)",0,196000,200000,400000
1,"(3, 11)",0,396000,400000,400000
2,"(4, 3)",0,190000,194000,400000
3,"(4, 3)",0,390000,394000,400000
4,"(5, 4)",0,184000,188000,400000
...,...,...,...,...,...
530,"(6, 7)",4,268400,304400,400000
531,"(7, 0)",9,311200,347200,400000
532,"(0, 7)",8,66000,74000,400000
533,"(8, 0)",6,38800,46800,400000
